In [ ]:
import numpy as np
from datetime import datetime
from telegram import *
from telegram.ext import *

print("I am ready to talk...")

# Some easy bot responses
def some_responses(input_text):
    user_message = str(input_text).lower()
    
    if user_message in ("hello", "hi", "hola"):
        return "Hey! How are you?"
    
    if user_message in ("good", "very well"):
        return "Fantastic!"
    
    if user_message in ("bad", "not good"):
        return "Cheer up! :)"
    
    if user_message in ("time", "time?"):
        now = datetime.now()
        date_time = now.strftime("%d/%m/%y, %H:%M:%S")
        return str(date_time)

    return "I don't understand you, try again please"

# Commands
def start_command(update, context):
    update.message.reply_text("My master is Bernat")

def help_command(update, context):
    update.message.reply_text("Try /random and input a vector. You can chose to compute the mean, variance or the sum of the vector")

def play_command(update, context):
    update.message.reply_text("New games coming soon... Stay tuned!!!")


def handle_message(update, context):
    text = str(update.message.text).lower() # Receives the text from user
    response = some_responses(text) # Process the text

    update.message.reply_text(response) # Replies the text

# Vandemonde matrix
def vand_command(update, context):
    try:
        n_1 = int(context.args[0])
        n_2 = int(context.args[1])
        n_3 = int(context.args[2])
        a = np.linspace(n_1, n_2, n_3)
        b = np.vander(a, 3)
        update.message.reply_text("The Vandermonde matrix is "+str(b))

    except (ValueError):
        update.message.reply_text("Use numbers please")

###
# Main command
def random_command(update, context):
    args = np.array(context.args).astype(np.float) # Convert to array of floats
    mean_ = np.mean(args) # Mean calculation
    var_ = np.var(args) # Variance calculation
    sum_ = np.sum(args) # Sum calculation

    reply_buttons = InlineKeyboardMarkup([
        [InlineKeyboardButton("Mean", callback_data="The mean is %s" % str(mean_))],
        [
            InlineKeyboardButton("Variance", callback_data="The variance is %s" % str(var_)),
            InlineKeyboardButton("Sum", callback_data="The sum is %s" % str(sum_)),
        ]
    ])
    update.message.reply_text(
        f'Hello {update.effective_user.first_name}, please choose an option:',
        reply_markup=reply_buttons
    )

def button(update, context):
    # Must call answer!
    update.callback_query.answer()
    # Remove buttons
    update.callback_query.message.edit_reply_markup(
        reply_markup=InlineKeyboardMarkup([])
    )
    update.callback_query.message.reply_text(update.callback_query.data)
###

def error(update, context):
    print(f"Update {update} caused error {context.error}")

def main():
    # Start the bot
    updater = Updater("1625011781:AAEraMjv64nwxoFbfFGjKvHRpF3GrMfRYtE", use_context=True)
    # Get the dispatcher to register handlers
    dp = updater.dispatcher
    
    # Different commands
    dp.add_handler(CommandHandler("start", start_command))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(CommandHandler("play", play_command))
    dp.add_handler(CommandHandler("vand", vand_command)) 
    ###
    dp.add_handler(CommandHandler("random", random_command))
    dp.add_handler(CallbackQueryHandler(button))
    ###

    dp.add_handler(MessageHandler(Filters.text, handle_message))

    # log the errors
    dp.add_error_handler(error)

    updater.start_polling()
    updater.idle()

main()

